<a href="https://colab.research.google.com/github/linlih/CovidFaceMaskDetector/blob/master/Covid_Face_Mask_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 下载数据

数据来源：https://github.com/X-zhangyang/Real-World-Masked-Face-Dataset


In [0]:
from pathlib import Path

import pandas as pd
from google_drive_downloader import GoogleDriveDownloader as gdd
from tqdm import tqdm

In [2]:
datasetPath = Path('./data/mask.zip')
# 从GoogleDrive的共享文件中下载训练数据
gdd.download_file_from_google_drive(file_id='1UlOk6EtiaXTHylRUx2mySgvJX9ycoeBp',
                  dest_path=str(datasetPath),
                  unzip=True)

Unzipping...Done.


In [0]:
datasetPath.unlink() # 删除下载的zipwe文件

In [4]:
# 构建DataFrame，并保存序列化，如果序列化过了，就无需无需执行这个内容，直接读入序列化的文件即可
# 注意DataFrame的append是要赋值等号的形式：maskDF = maskDF.append(xxx)，这个使用形式和其他直接append无法赋值就生效的不一致，要十分注意
datasetPath = Path('./data/self-built-masked-face-recognition-dataset')
maskPath = datasetPath/'AFDB_masked_face_dataset'
nonMaskPath = datasetPath/'AFDB_face_dataset'

maskDF = pd.DataFrame()

for subject in tqdm(list(maskPath.iterdir()),desc='mask photos'):
  for imgPath in subject.iterdir():
    maskDF = maskDF.append({
        'image': str(imgPath),
        'mask': 1
    }, ignore_index=True)

for subject in tqdm(list(nonMaskPath.iterdir()),desc='no mask photos'):
  for imgPath in subject.iterdir():
    maskDF = maskDF.append({
        'image': str(imgPath),
        'mask': 0
    }, ignore_index=True)
    
dfName = './data/mask_df.pickle'
print(f'saving DataFrame to {dfName}')
maskDF.to_pickle(dfName) # 保存序列化文件，读取的函数使用pd.read_pickle

no mask photos: 100%|██████████| 460/460 [04:33<00:00,  1.68it/s]

saving DataFrame to ./data/mask_df.pickle


In [0]:
# 如果已经序列化过，直接执行这个创建DataFrame即可
maskDF = pd.read_pickle('./data/mask_df.pickle')

In [0]:
# 统计结果中共有戴口罩的人脸图片是2203张，正常人脸是90468张
# 和Github数据集上说明的5千张戴口罩和9万张正常人脸有一定的出入
maskDF['mask'].value_counts()

In [0]:
# 构建Dataset，这里是为了能够让PyTorch进行读取
import cv2
from torch import long, tensor
from torch.utils.data.dataset import Dataset
from torchvision.transforms import Compose, Resize, ToPILImage, ToTensor

In [0]:
class MaskDataset(Dataset):
  def __init__(self, dataFrame):
    self.dataFrame = dataFrame
    self.transformations = Compose([
        ToPILImage(),
        Resize((100, 100)), # 每张人脸的大小调整为100*100
        ToTensor(),
    ])
    
  def __getitem__(self, key):
    if isinstance(key, slice):
      raise NotImplementedError('slicing is not supported')
    row = self.dataFrame.iloc[key]
    return {
        'image': self.transformations(cv2.imread(row['image'])),
        'mask': tensor([row['mask']], dtype=long)
    }

  def __len__(self):
    return len(self.dataFrame.index)

In [7]:
!pip install pytorch-lightning -q

     |████████████████████████████████| 256kB 8.3MB/s 
     |████████████████████████████████| 829kB 16.5MB/s 


In [0]:
# 构建模型
from pathlib import Path
from typing import Dict, List, Union

import pandas as pd
import pytorch_lightning as pl
import torch
import torch.nn.init as init

from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from torch import Tensor
from torch.nn import (Conv2d, CrossEntropyLoss, Linear, MaxPool2d, ReLU, Sequential)
from torch.optim import Adam
from torch.optim.optimizer import Optimizer
from torch.utils.data import DataLoader

In [0]:
class MaskDetector(pl.LightningModule):
  def __init__(self, maskDFPath: Path=None):
    super(MaskDetector, self).__init__()
    self.maskDFPath = maskDFPath
    self.maskDF = None
    self.trainDF = None
    self.validationDF = None
    self.crossEntropyLoss = None
    self.learningRate = 0.00001

    self.convLayer1 = convLayer1 = Sequential(Conv2d(3, 32, kernel_size=(3, 3), padding=(1, 1)),
                           ReLU(),
                           MaxPool2d(kernel_size=(2, 2))
                           )
    self.convLayer2 = convLayer2 = Sequential(Conv2d(32, 64, kernel_size=(3, 3), padding=(1, 1)),
                           ReLU(),
                           MaxPool2d(kernel_size=(2, 2))
                           )
    self.convLayer3 = convLayer3 = Sequential(Conv2d(64, 128, kernel_size=(3, 3), padding=(1, 1), stride=(3, 3)),
                           ReLU(),
                           MaxPool2d(kernel_size=(2, 2))
                           )
    self.linearLayers = linearLayers = Sequential(Linear(in_features=2048, out_features=1024),
                            ReLU(),
                            Linear(in_features=1024, out_features=2))
    
    for sequential in [convLayer1, convLayer2, convLayer3, linearLayers]:
      for layer in sequential.children():
        if isinstance(layer, (Linear, Conv2d)):
          init.xavier_uniform_(layer.weight)
  
  def forward(self, x: Tensor):
    out = self.convLayer1(x)
    out = self.convLayer2(out)
    out = self.convLayer3(out)
    out = out.view(-1, 2048)
    out = self.linearLayers(out)
    return out
  
  def prepare_data(self) -> None:
    self.maskDF = maskDF = pd.read_pickle(self.maskDFPath)
    train, validate = train_test_split(maskDF, test_size=0.3, random_state=0, stratify=maskDF['mask'])
    self.trainDF = MaskDataset(train)
    self.validateDF = MaskDataset(validate)

    maskNum = maskDF[maskDF['mask'] == 1].shape[0]
    nonMaskNum = maskDF[maskDF['mask'] == 0].shape[0]
    nSamples = [nonMaskNum, maskNum]
    normedWeights = [1 - (x/sum(nSamples)) for x in nSamples]
    self.crossEntropyLoss = CrossEntropyLoss(weight=torch.tensor(normedWeights))

  def train_dataloader(self) -> DataLoader:
    return DataLoader(self.trainDF, batch_size=32, shuffle=True, num_workers=4)

  def val_dataloader(self) -> DataLoader:
    return DataLoader(self.validateDF, batch_size=32, num_workers=4)
  
  def configure_optimizers(self) -> Optimizer:
    return Adam(self.parameters(), lr=self.learningRate)

  def training_step(self, batch: dict, _batch_idx: int) -> Dict[str, Tensor]:
    inputs, labels = batch['image'], batch['mask']
    labels = labels.flatten()
    outputs = self.forward(inputs)
    loss = self.crossEntropyLoss(outputs, labels)
    
    tensorboardLogs = {'train_loss': loss}
    return {'loss': loss, 'log': tensorboardLogs}

  def validation_step(self, batch:dict, _batch_idx: int) -> Dict[str, Tensor]:
    inputs, labels = batch['image'], batch['mask']
    labels = labels.flatten()
    outputs = self.forward(inputs)
    loss = self.crossEntropyLoss(outputs, labels)

    _, outputs = torch.max(outputs, dim=1)
    valAcc = accuracy_score(outputs.cpu(), labels.cpu())
    valAcc = torch.tensor(valAcc)
    return {'val_loss': loss, 'val_acc': valAcc}
  
  def validation_epoch_end(self, outputs: List[Dict[str, Tensor]]) \
     -> Dict[str, Union[Tensor, Dict[str, Tensor]]]:
    avgLoss = torch.stack([x['val_loss'] for x in outputs]).mean()
    avgAcc = torch.stack([x['val_acc'] for x in outputs]).mean()
    tensorboardLogs = {'val_loss': avgLoss, 'val_acc': avgAcc}
    return {'val_loss':avgLoss, 'log': tensorboardLogs}

In [10]:
# colab在这里训练会卡死
model = MaskDetector(Path('./data/mask_df.pickle'))

checkpoint_callback = ModelCheckpoint(
    filepath = './checkpoints/weights.ckpt',
    save_weights_only=True,
    verbose=True,
    monitor='val_acc',
    mode='max'
)
trainer = Trainer(gpus=1,
          max_epochs=10,
          checkpoint_callback=checkpoint_callback,
          profiler=True)
trainer.fit(model)

GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]

   | Name             | Type             | Params
--------------------------------------------------
0  | convLayer1       | Sequential       | 896   
1  | convLayer1.0     | Conv2d           | 896   
2  | convLayer1.1     | ReLU             | 0     
3  | convLayer1.2     | MaxPool2d        | 0     
4  | convLayer2       | Sequential       | 18 K  
5  | convLayer2.0     | Conv2d           | 18 K  
6  | convLayer2.1     | ReLU             | 0     
7  | convLayer2.2     | MaxPool2d        | 0     
8  | convLayer3       | Sequential       | 73 K  
9  | convLayer3.0     | Conv2d           | 73 K  
10 | convLayer3.1     | ReLU             | 0     
11 | convLayer3.2     | MaxPool2d        | 0     
12 | linearLayers     | Sequential       | 2 M   
13 | linearLayers.0   | Linear           | 2 M   
14 | linearLayers.1   | ReLU             | 0     
15 | linearLayers.2   | Linear   


Epoch 00000: val_acc reached 0.96223 (best 0.96223), saving model to ./checkpoints/_ckpt_epoch_0.ckpt as top 1
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: Did not find hyperparameters at model hparams. Saving checkpoint without hyperparameters.
  warnings.warn(*args, **kwargs)



Epoch 00001: val_acc reached 0.97460 (best 0.97460), saving model to ./checkpoints/_ckpt_epoch_1.ckpt as top 1



Epoch 00002: val_acc  was not in top 1



Epoch 00003: val_acc reached 0.99014 (best 0.99014), saving model to ./checkpoints/_ckpt_epoch_3.ckpt as top 1


...


Epoch 00006: val_acc  was not in top 1



Epoch 00007: val_acc  was not in top 1



Epoch 00008: val_acc reached 0.99227 (best 0.99227), saving model to ./checkpoints/_ckpt_epoch_8.ckpt as top 1



Epoch 00009: val_acc  was not in top 1


Profiler Report

Action              	|  Mean duration (s)	|  Total time (s) 
-----------------------------------------------------------------
on_train_start      	|  0.051214       	|  0.051214       
on_epoch_start      	|  0.0025172      	|  0.025172       
get_train_batch     	|  0.006346       	|  128.76         
on_batch_start      	|  1.4932e-05     	|  0.30282        
model_forward       	|  0.0058799      	|  119.24         
model_backward      	|  0.0040581      	|  82.298         
on_after_backward   	|  3.2854e-06     	|  0.066628       
optimizer_step      	|  0.0027456      	|  55.681         
on_batch_end        	|  0.010115       	|  205.12         
on_epoch_end        	|  1.9402e-05     	|  0.00019402     
on_train_end        	|  0.0042903      	|  0.0042903      



1